In [83]:
import os
import json
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery
from azure.search.documents import SearchClient
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os
import requests
import base64

from dotenv import load_dotenv

load_dotenv('.env', override=True)

True

In [84]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AOAI_ENDPOINT"), 
    api_key=os.getenv("AOAI_KEY"),  
    api_version="2024-05-01-preview"
)

gpt_model_deployment = os.getenv("AOAI_GPT_MODEL")
embedding_model = os.getenv("AOAI_EMBEDDING_MODEL")

In [85]:
def prompt_llm(sys_message, user_message, gpt_model_deployment, tokens, temperature, chat_history=[]):
    client = AzureOpenAI(
        azure_endpoint=os.environ['AOAI_ENDPOINT'], api_key=os.environ['AOAI_KEY'], api_version="2024-02-15-preview"
    )
    messages = [{"role": "system", "content": sys_message}] + chat_history + [{"role": "user", "content": user_message}]
    completion = client.chat.completions.create(
    model=gpt_model_deployment,
    messages=messages,
    max_tokens = tokens,
    temperature = temperature
    )
    return completion.choices[0].message.content, completion.usage.prompt_tokens

In [86]:
sys_msg = f"""You help people develop agent-based workflows based on their workflow process descriptions.
You will be provided with an image showcasing a business process.

1.) First, you should identify what agents are needed to replicate this process.
You can do this by using the `get_agents()` tool you have access to.

2.) Then you need to retrieve a script template that can be used to create the agentic workflow.
You can do this by using the `retrieve_template_script()` tool you have access to.

3.) Then, you will create an agentic workflow based on the process description and the script template.
You can do this by using the `create_agentic_workflow()` tool you have access to.

4.) Finally, you will submit the agentic workflow to the client for review and corrections.
You can do this by using the `review_agentic_workflow()` tool you have access to.

By using these tools, in the order described above, you will complete the task successfully.
"""

In [87]:
def get_agents(img_path):
    with open(img_path, "rb") as image_file:
        img = base64.b64encode(image_file.read()).decode('utf-8')
    sys_msg = """
    You help people develop agent-based workflows based on their process descriptions. 
    Each node in the workflow should represent an agent. 
    Describe these agents and what their goals are.
    Describe the workflow process for how the agents should be used.
    """
    messages = [{"role": "system", "content": sys_msg}]
    user_content = {
        "role": "user",
        "content": [
            {
            "type": "image_url",
            "image_url": {
                    "url": f"data:image/jpeg;base64,{img}"
                     , "detail": "high"
                }
            }  
        ]
    }
    messages.append(user_content)
    completion = client.chat.completions.create(
        model=gpt_model_deployment,
        messages=messages,
        max_tokens = 2000,
        temperature = 0.0
    )
    api_base = os.environ['AOAI_ENDPOINT']
    api_key = os.environ['AOAI_KEY']
    deployment_name = os.environ['AOAI_GPT_MODEL']

    base_url = f"{api_base}openai/deployments/{deployment_name}" 
    headers = {   
        "Content-Type": "application/json",   
        "api-key": api_key 
    } 
    endpoint = f"{base_url}/chat/completions?api-version=2023-12-01-preview" 
    data = { 
        "messages": messages, 
        "temperature": 0.0,
        "top_p": 0.95,
        "max_tokens": 2048
    }   
    response = requests.post(endpoint, headers=headers, data=json.dumps(data)) 
    resp_str = response.json()['choices'][0]['message']['content']
    return resp_str

def retrieve_template_script():
    return '../templates/functionapp.py'

def create_agentic_workflow(agents, dir_name, template_script_path):
    with open(template_script_path, 'r') as f:
        template_script = f.read()
    sys_msg = f"""
    You help people create agentic workflows using Azure Durable Functions.
    You will be provided with a list of agents, and a sequence in which they should be executed.
    You will also be provided with a durable function template.
    Use this template to create an agent workflow.
    The run_agent_orchestrator function should be used for agent orchestration.
    Each call to an agent should be done using a call_activity function.
    Your code should include activities for each agent. 
    If there is conditional logic in the agent workflow, include this in the agent activity function.

    YOUR RESPONSE SHOULD CONSIST OF ONLY PYTHON CODE.

    ### TEMPLATE: {template_script}
    """
    usr_msg = f'## AGENTS DESCRIPTION: {agents}'
    messages = [{"role": "system", "content": sys_msg}, {"role": "user", "content": usr_msg}]
    completion = client.chat.completions.create(
        model=gpt_model_deployment,
        messages=messages,
        max_tokens = 4000,
        temperature = 0.0
    )
    os.makedirs(dir_name, exist_ok=True)
    with open(f'{dir_name}/functionapp.py', 'w') as f:
        f.write(completion.choices[0].message.content.replace('```python', '').replace('```', ''))
    
    return f'{dir_name}/functionapp.py'

def review_agentic_workflow(template_script_path):
    with open(template_script_path, 'r') as f:
        template_script = f.read()
    sys_msg = f"""
    You will review and improve an agentic workflow built using Azure Durable Functions.
    Review the code and move any loops in the main orchestrator function, to a separate activity function.
    Ensure that the code is clean and follows best practices.

    YOUR RESPONSE SHOULD CONSIST OF ONLY PYTHON CODE.
    """
    usr_msg = f'## DURABLE FUNCTIONS AGENTIC WORKFLOW: {template_script}'
    messages = [{"role": "system", "content": sys_msg}, {"role": "user", "content": usr_msg}]
    completion = client.chat.completions.create(
        model=gpt_model_deployment,
        messages=messages,
        max_tokens = 4000,
        temperature = 0.0
    )
    with open(template_script_path, 'w') as f:
        f.write(completion.choices[0].message.content.replace('```python', '').replace('```', ''))
    
    return completion.choices[0].message.content
    

In [88]:
def run_conversation(sys_msg, img, dir_name, history=[]):
    if len(history)==0:
        messages = [{"role": "system", "content": sys_msg}, {"role": "user", "content": 'Workflow image at ' + img}]
    else:
        pass
    tools = [  
    {  
        "type": "function",  
        "function": {  
            "name": "get_agents",  
            "description": "Interprets an image of a business process workflow diagram and creates a description of a set of agents that could complete the task",  
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "img": {  
                        "type": "string",  
                        "description": "Path to the image of the business process workflow diagram"  
                    }  
                },  
                "required": ["img"]  
            }  
        }  
    },  
    {  
        "type": "function",  
        "function": {  
            "name": "retrieve_template_script",  
            "description": "Retrieve a path to a sample Azure Durable Function python script to be used as the basis for a function-based agent workflow"  
        }  
    },  
    {  
        "type": "function",  
        "function": {  
            "name": "generate_agentic_workflow",  
            "description": "Develops an Azure Durable Function based agentic workflow using provided agent descriptions, and a template script. Saves the file and returns the path.",  
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "agents_description": {  
                        "type": "string",  
                        "description": "A textual description of all agents and their tasks"  
                    },  
                    "dir_name": {  
                        "type": "string",  
                        "description": "Directory name where the generated workflow will be saved"  
                    },  
                    "template_script_path": {  
                        "type": "string",  
                        "description": "A path to a template script to be used as the basis for the workflow"  
                    }  
                },  
                "required": ["agents_description", "dir_name", "template_script_path"]  
            }  
        }  
    },
    {  
        "type": "function",  
        "function": {  
            "name": "review_agentic_workflow",  
            "description": "Reviews an Azure Durable Function based agentic workflow and makes revisions as needed",  
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "script_path": {  
                        "type": "string",  
                        "description": "The path to the Azure Durable Function agentic workflow script to be reviewed"  
                    }  
                },  
                "required": ["script_path"]  
            }  
        }  
    }  
]  

    # First API call: Ask the model to use the functions
    response = client.chat.completions.create(
        model=gpt_model_deployment,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0.0
    )

    response_message = response.choices[0].message
    messages.append(response_message)

    is_done = False

    while not is_done:
        # Handle function calls
        if response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function call: {function_name}")  
                print(f"Function arguments: {function_args}")  
                
                if function_name == "get_agents":
                    function_response = get_agents(function_args['img'])
                    print(function_response)
                elif function_name == "retrieve_template_script":
                    function_response = retrieve_template_script()
                    print(function_response)
                elif function_name == "generate_agentic_workflow":
                    print(function_args['template_script_path'])
                    function_response = create_agentic_workflow(
                        agents=function_args["agents_description"],
                        dir_name=dir_name,
                        template_script_path=function_args["template_script_path"]
                    )
                    print(function_response)
                elif function_name == "review_agentic_workflow":
                    function_response = review_agentic_workflow(
                        template_script_path=function_args["script_path"]
                    )
                    print(function_response)
                else:
                    function_response = json.dumps({"error": "Unknown function"})
                
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                })
        else:
            print("No tool calls were made by the model.")  

        # Second API call: Get the final response from the model
        response = client.chat.completions.create(
            model=gpt_model_deployment,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.0
        )

        response_message = response.choices[0].message
        messages.append(response_message)
        if response_message.tool_calls:
            pass
        else:
            is_done = True
    

    return response_message.content

In [91]:
run_conversation(sys_msg, '../img/eclflow.png', '../ecl-src')

Function call: get_agents
Function arguments: {'img': '../img/eclflow.png'}
This flowchart outlines a process for analyzing event logs related to pH probe calibrations, power-up resets, and sensor alarms. Here is a breakdown of the agents and their goals, as well as the workflow process:

### Agents and Their Goals

1. **Data Collection Agent**
   - **Goal:** Obtain the last 30 days of data for analysis for a particular controller.
   
2. **Calibration Check Agent**
   - **Goal:** Determine if calibration has been done recently.
   
3. **Event Log Analysis Agent**
   - **Goal:** Check for specific events in the logs, such as 1-point and 2-point probe calibrations, power-up resets, and sensor alarms.
   
4. **Response Agent**
   - **Goal:** Send appropriate responses based on the analysis of the event logs.

### Workflow Process

1. **Start**
   - The process begins.

2. **Data Collection Agent**
   - **Action:** Obtain the last 30 days of data for analysis for a particular controller.


'The agentic workflow has been successfully created and reviewed. Here is the generated script for your workflow:\n\n```python\nimport azure.functions as func\nimport azure.durable_functions as df\nimport logging\nimport json\nimport os\n\napp = df.DFApp(http_auth_level=func.AuthLevel.FUNCTION)\n\n# An HTTP-Triggered Function with a Durable Functions Client binding\n@app.route(route="orchestrators/{functionName}")\n@app.durable_client_input(client_name="client")\nasync def http_start(req: func.HttpRequest, client):\n    function_name = req.route_params.get(\'functionName\')\n    payload = json.loads(req.get_body())\n    instance_id = await client.start_new(function_name, client_input=payload)\n    response = client.create_check_status_response(req, instance_id)\n    return response\n\n# Orchestrator\n@app.orchestration_trigger(context_name="context")\ndef agent_orchestrator(context):\n\n    # Get the input payload from the context\n    payload = context.get_input()\n\n    # Data Collec